In [4]:
import unittest
import numpy as np
import importlib
import sys
sys.path.append('../')
from src import pauli_basis_class
importlib.reload(pauli_basis_class)
from src.pauli_basis_class import Pauli_Basis
import math

In [22]:
class Pauli_Basis_TestCase(unittest.TestCase):
    def test_get_correct_dimensions(self):
        n = 2
        my_pauli_basis = Pauli_Basis(n)
        result = my_pauli_basis.dimension
        self.assertEqual(result, n)

    def test_only_powers_of_2(self):
        n = 3
        with self.assertRaises(Exception) as context:
            Pauli_Basis(n)  

        self.assertEqual(str(context.exception), "Must be a power of 2")

    def test_tesnor_product_list_of_arrays(self):
        n = 2 
        my_pauli_basis = Pauli_Basis(n)

        list_1 = [np.array([[1,1,3],[3,2,1],[4,5,6]]), np.array([[4,5,7],[8,4,6],[1j,6,-6]])]
        list_2 = [np.array([[4,5,8],[8,8,9],[9,3,5]])] +  list_1
        
        result = my_pauli_basis.tensor_product_list(list_1,list_2) 

        expected_list_size = len(list_1)*len(list_2)
        expected_matrix_size = len(np.kron(list_1[0], list_2[0])) 
        self.assertEqual(expected_list_size, len(result))
        self.assertEqual(expected_matrix_size, len(result[0])) 
        self.assertTrue(np.array_equal(np.kron(list_1[-1],list_2[-1]), result[-1]))      

    def test_recursive_tensor_prod_list(self):

        n = 4
        my_pauli_basis = Pauli_Basis(n)
        list_1 = [np.array([[1,1,3],[3,2,1],[4,5,6]]), np.array([[4,5,7],[8,4,6],[1j,6,-6]])]
        list_2 = [np.array([[4,5,8],[8,8,9],[9,3,5]]), np.array([[1,2,3],[4,5,6],[7,8,9]])]
        number_of_tensor_products = 3

        result = my_pauli_basis.recursive_tensor_product_list(list_1, list_2, number_of_tensor_products)

        expected_list_size = len(list_1)*(len(list_2)**(number_of_tensor_products))
        expected_matrix_size = len(list_1[0])*len(list_2[0])**(number_of_tensor_products)

        self.assertEqual(expected_list_size, len(result))
        self.assertEqual(expected_matrix_size, len(result[0]))

    def test_pauli_basis_su_2_without_coefficients(self):
        n = 2
        my_pauli_basis = Pauli_Basis(n)

        result = my_pauli_basis.pauli_basis_matrices_without_coefficient()
        
        expected_list = [np.array([[0,1],[1,0]]), 
                         np.array([[0,-1j],[1j, 0]]), 
                         np.array([[1,0],[0,-1]])]
        
        self.assertTrue(all(np.array_equal(result[i], expected_list[i]) for i in range(len(result))))    

    
    def test_pauli_basis_matrices_creates_correct_number_of_matrices_with_correct_dimension(self):

        n = 16
        my_pauli_basis = Pauli_Basis(n)

        expected_number_of_matrices = n**2 - 1
        expected_matrix_size = n
        
        output_basis_list = my_pauli_basis.pauli_basis_matrices_without_coefficient()
        
        actual_number_of_matrices = len(output_basis_list)
        actual_matrix_size = len(output_basis_list[0])

        self.assertEqual(expected_number_of_matrices, actual_number_of_matrices)
        self.assertEqual(expected_matrix_size, actual_matrix_size)


    def test_find_correct_coefficient_for_the_basis_matrices(self):
        n = 2 
        my_pauli_basis = Pauli_Basis(n)
        
        result = my_pauli_basis.coefficient()
        expected = 1/np.sqrt(n)
        self.assertAlmostEqual(result, expected)


    def test_getter_for_pauli_basis_with_coefficients(self):
        n = 2
        my_pauli_basis = Pauli_Basis(n)
        result = my_pauli_basis.basis
        
        expected_list = [
                        (1/np.sqrt(2))*np.array([[0,1],[1,0]]),
                        (1/np.sqrt(2))*np.array([[0,-1j],[1j, 0]]), 
                        (1/np.sqrt(2))*np.array([[1,0],[0,-1]])]

        self.assertTrue(all(np.allclose(result[i], expected_list[i]) for i in range(len(result))))

    def test_orthonormality_of_basis_matrices(self):

        n = 4
        my_pauli_basis = Pauli_Basis(n)

        basis_list = my_pauli_basis.basis

        result_list = []

        for i in range(n**2 - 1):
            for j in range(n**2 - 1):
                value = np.trace(np.dot(basis_list[i],basis_list[j]))

                if i == j:
                    if math.isclose(value, 1):
                        result_list.append(True)
                    else:
                        result_list.append(False)
                else:
                    if value == 0:
                        result_list.append(True)
                    else:
                        result_list.append(False)
        self.assertTrue(all(result_list))

    # def test_find_cadidates_masks_for_input_matrix_from_basis_list(self):
        
    #     n = 2
    #     my_pauli_basis  = Pauli_Basis(n)

    #     test_matrix = np.array([[0,5],[5,0]])
    #     output = my_pauli_basis.find_candidate_masks_from_basis(input_matrix = test_matrix)
    #     #just outputting the list of 
    #     expected_list = [0, 1]

    #     self.assertEqual(output, expected_list)


In [23]:
unittest.main(argv=[''], exit=False)

....C:\Users\waqas\AppData\Local\Temp\ipykernel_29628\17800082.py:111: ComplexWarning: Casting complex values to real discards the imaginary part
  if math.isclose(value, 1):
.....
----------------------------------------------------------------------
Ran 9 tests in 1.259s

OK
